# Phase 4 Project Notebook
- Author: Jonathan Holt
- Data Science Flex

## Business Problem
- What are the top 5 best zip codes for us to invest in?

## Questions to Answer

## What Models & Metrics I plan on using

## Helper Functions
- Functions provided by Flatiron

In [1]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [2]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

## Loading Data

In [93]:
import pandas as pd

pd.set_option('display.max_rows', 1000) #change the amount of rows displayed

In [4]:
df = pd.read_csv("zillow_data.csv")

In [5]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [26]:
df.describe()

,RegionID,RegionName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
count,14723.000000,14723.000000,14723.000000,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,...,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04
mean,81075.010052,48222.348706,7362.000000,1.182991e+05,1.184190e+05,1.185374e+05,1.186531e+05,1.187803e+05,1.189275e+05,1.191205e+05,...,2.733354e+05,2.748658e+05,2.764646e+05,2.780332e+05,2.795209e+05,2.810953e+05,2.826571e+05,2.843687e+05,2.865114e+05,2.880399e+05
std,31934.118525,29359.325439,4250.308342,8.600251e+04,8.615567e+04,8.630923e+04,8.646795e+04,8.665094e+04,8.687208e+04,8.715185e+04,...,3.603984e+05,3.614678e+05,3.627563e+05,3.644610e+05,3.656003e+05,3.670454e+05,3.695727e+05,3.717739e+05,3.724612e+05,3.720544e+05
min,58196.000000,1001.000000,1.000000,1.130000e+04,1.150000e+04,1.160000e+04,1.180000e+04,1.180000e+04,1.200000e+04,1.210000e+04,...,1.440000e+04,1.450000e+04,1.470000e+04,1.480000e+04,1.450000e+04,1.430000e+04,1.410000e+04,1.390000e+04,1.380000e+04,1.380000e+04
25%,67174.500000,22101.500000,3681.500000,6.880000e+04,6.890000e+04,6.910000e+04,6.920000e+04,6.937500e+04,6.950000e+04,6.960000e+04,...,1.269000e+05,1.275000e+05,1.282000e+05,1.287000e+05,1.292500e+05,1.299000e+05,1.306000e+05,1.310500e+05,1.319500e+05,1.324000e+05
50%,78007.000000,46106.000000,7362.000000,9.950000e+04,9.950000e+04,9.970000e+04,9.970000e+04,9.980000e+04,9.990000e+04,9.995000e+04,...,1.884000e+05,1.896000e+05,1.905000e+05,1.914000e+05,1.925000e+05,1.934000e+05,1.941000e+05,1.950000e+05,1.967000e+05,1.981000e+05
75%,90920.500000,75205.500000,11042.500000,1.432000e+05,1.433000e+05,1.432250e+05,1.432250e+05,1.435000e+05,1.437000e+05,1.439000e+05,...,3.050000e+05,3.066500e+05,3.085000e+05,3.098000e+05,3.117000e+05,3.134000e+05,3.151000e+05,3.168500e+05,3.188500e+05,3.211000e+05
max,753844.000000,99901.000000,14723.000000,3.676700e+06,3.704200e+06,3.729600e+06,3.754600e+06,3.781800e+06,3.813500e+06,3.849600e+06,...,1.888990e+07,1.870350e+07,1.860530e+07,1.856940e+07,1.842880e+07,1.830710e+07,1.836590e+07,1.853040e+07,1.833770e+07,1.789490e+07


In [7]:
#get_datetimes(df)

## Checking for Null Values

In [22]:
df.isnull().sum()

RegionID         0
RegionName       0
City             0
State            0
Metro         1043
CountyName       0
SizeRank         0
1996-04       1039
1996-05       1039
1996-06       1039
1996-07       1039
1996-08       1039
1996-09       1039
1996-10       1039
1996-11       1039
1996-12       1039
1997-01       1039
1997-02       1039
1997-03       1039
1997-04       1039
1997-05       1039
1997-06       1039
1997-07       1038
1997-08       1038
1997-09       1038
1997-10       1038
1997-11       1038
1997-12       1038
1998-01       1036
1998-02       1036
1998-03       1036
1998-04       1036
1998-05       1036
1998-06       1036
1998-07       1036
1998-08       1036
1998-09       1036
1998-10       1036
1998-11       1036
1998-12       1036
1999-01       1036
1999-02       1036
1999-03       1036
1999-04       1036
1999-05       1036
1999-06       1036
1999-07       1036
1999-08       1036
1999-09       1036
1999-10       1036
1999-11       1036
1999-12       1036
2000-01     

### Analysis

There are many Null Values. 
- For categorical data, 7% of Metro are null (1043 of 14,723).
- Dates from 1996 - mid 2003, also have 7% null values.
- Then it starts to get better. 6% null and decreasing.

What is my decision on null values?
- keep?
- delete?
- change (mean)?

## Analysis of Categories

In [8]:
cat_df = df.iloc[:,0:7]
cat_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank
0,84654,60657,Chicago,IL,Chicago,Cook,1
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2
2,91982,77494,Katy,TX,Houston,Harris,3
3,84616,60614,Chicago,IL,Chicago,Cook,4
4,93144,79936,El Paso,TX,El Paso,El Paso,5


In [54]:
cat_df = cat_df.astype('category')
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   RegionID    14723 non-null  category
 1   RegionName  14723 non-null  category
 2   City        14723 non-null  category
 3   State       14723 non-null  category
 4   Metro       13680 non-null  category
 5   CountyName  14723 non-null  category
 6   SizeRank    14723 non-null  category
dtypes: category(7)
memory usage: 2.8 MB


In [55]:
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   RegionID    14723 non-null  category
 1   RegionName  14723 non-null  category
 2   City        14723 non-null  category
 3   State       14723 non-null  category
 4   Metro       13680 non-null  category
 5   CountyName  14723 non-null  category
 6   SizeRank    14723 non-null  category
dtypes: category(7)
memory usage: 2.8 MB


In [78]:
print("RegionID unique values:", cat_df['RegionID'].nunique())
print("RegionName unique values:", cat_df['RegionName'].nunique())
print("City unique values:", cat_df['City'].nunique())
print("State unique values:", cat_df['State'].nunique())
print("Metro unique values:", cat_df['Metro'].nunique())
print("CountyName unique values:", cat_df['CountyName'].nunique())
print("SizeRank unique values:", cat_df['SizeRank'].nunique())

RegionID unique values: 14723
RegionName unique values: 14723
City unique values: 7554
State unique values: 51
Metro unique values: 701
CountyName unique values: 1212
SizeRank unique values: 14723


In [87]:
cat_df.sort_values(by = ['RegionName'], ascending = True).head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank
5850,58196,1001,Agawam,MA,Springfield,Hampden,5851
4199,58197,1002,Amherst,MA,Springfield,Hampshire,4200
11213,58200,1005,Barre,MA,Worcester,Worcester,11214
6850,58201,1007,Belchertown,MA,Springfield,Hampshire,6851
14547,58202,1008,Blandford,MA,Springfield,Hampden,14548


A google search shows that RegionName is the ZipCode for each Region. However, upon sorting, I discovered that any ZipCode beginning with a 0 was ignoring it and displaying as a 4 digit number. I will use the .str().zfill() method to ensure that all RegionNames are displaying the as 5 digits.

In [90]:
cat_df['RegionName'] = cat_df['RegionName'].astype(str).str.zfill(5)

In [95]:
cat_df.sort_values(by = ['RegionName'], ascending = True).head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank
5850,58196,01001,Agawam,MA,Springfield,Hampden,5851
4199,58197,01002,Amherst,MA,Springfield,Hampshire,4200
11213,58200,01005,Barre,MA,Worcester,Worcester,11214
6850,58201,01007,Belchertown,MA,Springfield,Hampshire,6851
14547,58202,01008,Blandford,MA,Springfield,Hampden,14548


That seems to have fixed it!

## Analysis of Data Values

In [116]:
df['RegionID_copy'] = df['RegionID']

In [117]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,RegionID_copy
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600,84654
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,310000,312500,314100,315000,316600,318100,319600,321100,321800,90668
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,320600,320200,320400,320800,321200,321200,323000,326900,329900,91982
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000,84616
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119400,120000,120300,120300,120300,120300,120500,121000,121500,93144


In [118]:
data_df = df.iloc[:, 7:]
data_df.head()

,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,...,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,RegionID_copy
0,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,...,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600,84654
1,235700.0,236900.0,236700.0,235400.0,233300.0,230600.0,227300.0,223400.0,219600.0,215800.0,...,310000,312500,314100,315000,316600,318100,319600,321100,321800,90668
2,210400.0,212200.0,212200.0,210700.0,208300.0,205500.0,202500.0,199800.0,198300.0,197300.0,...,320600,320200,320400,320800,321200,321200,323000,326900,329900,91982
3,498100.0,500900.0,503100.0,504600.0,505500.0,505700.0,505300.0,504200.0,503600.0,503400.0,...,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000,84616
4,77300.0,77300.0,77300.0,77300.0,77400.0,77500.0,77600.0,77700.0,77700.0,77800.0,...,119400,120000,120300,120300,120300,120300,120500,121000,121500,93144


In [119]:
data_df.columns = pd.to_datetime(data_df.columns)
data_df.head()

ParserError: Unknown string format: RegionID_copy

In [99]:
# changing values to thousands for ease of reading
data_df = data_df.applymap(lambda x: x/1000)
data_df.head()

,1996-04-01,1996-05-01,1996-06-01,1996-07-01,1996-08-01,1996-09-01,1996-10-01,1996-11-01,1996-12-01,1997-01-01,...,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
0,334.2,335.4,336.5,337.6,338.5,339.5,340.4,341.3,342.6,344.4,...,1005.5,1007.5,1007.8,1009.6,1013.3,1018.7,1024.4,1030.7,1033.8,1030.6
1,235.7,236.9,236.7,235.4,233.3,230.6,227.3,223.4,219.6,215.8,...,308.0,310.0,312.5,314.1,315.0,316.6,318.1,319.6,321.1,321.8
2,210.4,212.2,212.2,210.7,208.3,205.5,202.5,199.8,198.3,197.3,...,321.0,320.6,320.2,320.4,320.8,321.2,321.2,323.0,326.9,329.9
3,498.1,500.9,503.1,504.6,505.5,505.7,505.3,504.2,503.6,503.4,...,1289.8,1287.7,1287.4,1291.5,1296.6,1299.0,1302.7,1306.4,1308.5,1307.0
4,77.3,77.3,77.3,77.3,77.4,77.5,77.6,77.7,77.7,77.8,...,119.1,119.4,120.0,120.3,120.3,120.3,120.3,120.5,121.0,121.5


In [101]:
data_df.head()

,1996-04-01,1996-05-01,1996-06-01,1996-07-01,1996-08-01,1996-09-01,1996-10-01,1996-11-01,1996-12-01,1997-01-01,...,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
0,334.2,335.4,336.5,337.6,338.5,339.5,340.4,341.3,342.6,344.4,...,1005.5,1007.5,1007.8,1009.6,1013.3,1018.7,1024.4,1030.7,1033.8,1030.6
1,235.7,236.9,236.7,235.4,233.3,230.6,227.3,223.4,219.6,215.8,...,308.0,310.0,312.5,314.1,315.0,316.6,318.1,319.6,321.1,321.8
2,210.4,212.2,212.2,210.7,208.3,205.5,202.5,199.8,198.3,197.3,...,321.0,320.6,320.2,320.4,320.8,321.2,321.2,323.0,326.9,329.9
3,498.1,500.9,503.1,504.6,505.5,505.7,505.3,504.2,503.6,503.4,...,1289.8,1287.7,1287.4,1291.5,1296.6,1299.0,1302.7,1306.4,1308.5,1307.0
4,77.3,77.3,77.3,77.3,77.4,77.5,77.6,77.7,77.7,77.8,...,119.1,119.4,120.0,120.3,120.3,120.3,120.3,120.5,121.0,121.5


In [103]:
data_df.describe().round(2)

,1996-04-01,1996-05-01,1996-06-01,1996-07-01,1996-08-01,1996-09-01,1996-10-01,1996-11-01,1996-12-01,1997-01-01,...,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
count,13684.0,13684.00,13684.00,13684.00,13684.00,13684.00,13684.00,13684.00,13684.00,13684.00,...,14723.00,14723.00,14723.00,14723.00,14723.00,14723.00,14723.00,14723.00,14723.00,14723.00
mean,118.3,118.42,118.54,118.65,118.78,118.93,119.12,119.35,119.69,120.12,...,273.34,274.87,276.46,278.03,279.52,281.10,282.66,284.37,286.51,288.04
std,86.0,86.16,86.31,86.47,86.65,86.87,87.15,87.48,87.91,88.41,...,360.40,361.47,362.76,364.46,365.60,367.05,369.57,371.77,372.46,372.05
min,11.3,11.50,11.60,11.80,11.80,12.00,12.10,12.20,12.30,12.50,...,14.40,14.50,14.70,14.80,14.50,14.30,14.10,13.90,13.80,13.80
25%,68.8,68.90,69.10,69.20,69.38,69.50,69.60,69.80,70.00,70.30,...,126.90,127.50,128.20,128.70,129.25,129.90,130.60,131.05,131.95,132.40
50%,99.5,99.50,99.70,99.70,99.80,99.90,99.95,100.10,100.30,100.60,...,188.40,189.60,190.50,191.40,192.50,193.40,194.10,195.00,196.70,198.10
75%,143.2,143.30,143.22,143.22,143.50,143.70,143.90,144.12,144.30,144.60,...,305.00,306.65,308.50,309.80,311.70,313.40,315.10,316.85,318.85,321.10
max,3676.7,3704.20,3729.60,3754.60,3781.80,3813.50,3849.60,3888.90,3928.80,3964.60,...,18889.90,18703.50,18605.30,18569.40,18428.80,18307.10,18365.90,18530.40,18337.70,17894.90


## Slicing out Years and attempting to Melt

In [110]:
yr_1996 = data_df.iloc[:,:9]
yr_1996.head()

,1996-04-01,1996-05-01,1996-06-01,1996-07-01,1996-08-01,1996-09-01,1996-10-01,1996-11-01,1996-12-01
0,334.2,335.4,336.5,337.6,338.5,339.5,340.4,341.3,342.6
1,235.7,236.9,236.7,235.4,233.3,230.6,227.3,223.4,219.6
2,210.4,212.2,212.2,210.7,208.3,205.5,202.5,199.8,198.3
3,498.1,500.9,503.1,504.6,505.5,505.7,505.3,504.2,503.6
4,77.3,77.3,77.3,77.3,77.4,77.5,77.6,77.7,77.7


In [111]:
melt_data(yr_1996)

KeyError: "The following 'id_vars' are not present in the DataFrame: ['City', 'CountyName', 'Metro', 'RegionID', 'RegionName', 'SizeRank', 'State']"

# Archive

In [106]:
#data_df.isnull().sum()

In [ ]:
#df.columns[7:]

In [ ]:
#date_time_cols = pd.to_datetime(df.columns[7:])
#date_time_cols

In [ ]:
#cat_cols = df.columns[:7]
#cat_cols

In [ ]:
#new_cols = cat_cols + date_time_cols
#new_cols

In [ ]:
#df.rename(columns[7:] = date_time_cols)

## Transposing DF

In [ ]:
flipped_df = df.transpose()

In [ ]:
flipped_df.head(10)

In [ ]:
#melt_data(flipped_df)
melt_data(df)

In [ ]:
#roi = table9.apply(lambda x: x['total_profit'] / x['production_budget'], axis=1)
test = df.loc[8]
#test

In [ ]:
melt_data(test)

## Exploring SizeRank

In [ ]:
df.sort_values('SizeRank').head()

In [ ]:
df.sort_values(by = ['State', 'City'], ascending = True)

In [ ]:
cat_data = df[['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank']]

In [ ]:
cat_data

In [ ]:
df.head()

In [ ]:
date_data = df.drop(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], axis=1)

In [ ]:
date_data.head()

In [ ]:
date_time_data = get_datetimes(date_data)

In [ ]:
date_time_data

In [ ]:
len(date_time_data)

In [ ]:
list(date_data.columns)

In [ ]:
#df2 = date_data.append(date_time_data)

## Changing column names to DateTime Format

In [ ]:
#df.columns = pd.to_datetime(df.columns)
date_data.columns = pd.to_datetime(date_data.columns)

In [ ]:
date_data

### Merging with Categories

In [ ]:
#imdb_df = pd.merge(table7, table4, on= 'tconst', how='inner')
base_df = pd.merge(cat_data, date_data, how='outer')

# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

## Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

### Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

## Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


## Step 1: Load the Data/Filtering for Chosen Zipcodes

## Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

## Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

## Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

## Step 5: ARIMA Modeling

## Step 6: Interpreting Results